In [34]:
import sys
print(sys.executable)

c:\Users\VAIBHAV VARSHNEY\anaconda3\python.exe


In [35]:
import os
print(os.getcwd())


c:\Users\VAIBHAV VARSHNEY\Desktop\Resume Analyzer\Notebooks


In [36]:
import pandas as pd
import json

# Load JSON Lines
data_path = '../DATA/master_resumes.jsonl'
# Each line is a JSON object
records = [json.loads(line) for line in open(data_path, 'r', encoding='utf-8')]

raw_df = pd.DataFrame(records)
print("Rows:", len(df))
raw_df.head(3)

Rows: 4817


,personal_info,experience,education,skills,projects,certifications,achievements,workshops,publications,teaching_experience,internships
0,"{'name': 'Unknown', 'email': 'Unknown', 'phone...","[{'company': 'Fresher', 'company_info': {'indu...","[{'degree': {'level': 'ME', 'field': 'Computer...",{'technical': {'programming_languages': [{'nam...,"[{'name': 'Unknown', 'description': 'Unknown',...",,NaN,NaN,NaN,NaN,NaN
1,"{'name': 'Unknown', 'email': 'Unknown', 'phone...","[{'company': 'Delta Controls, Dubai FZCO', 'co...","[{'degree': {'level': 'B.E', 'field': 'Electro...",{'technical': {'project_management': [{'name':...,"[{'name': 'FGP/WPMP', 'description': 'Led syst...",,NaN,NaN,NaN,NaN,NaN
2,"{'name': 'Not Provided', 'email': 'Not Provide...","[{'company': 'Parkar Consulting and Labs', 'co...","[{'degree': {'level': 'B.E.', 'field': 'Not Pr...",{'technical': {'programming_languages': [{'nam...,"[{'name': 'FPGA Implementation', 'description'...",,NaN,NaN,NaN,NaN,NaN


In [37]:
print(raw_df.columns)
print('INFO')
print(raw_df.info())
print('MISSING VALUES')
print(raw_df.isnull().sum())


Index(['personal_info', 'experience', 'education', 'skills', 'projects',
       'certifications', 'achievements', 'workshops', 'publications',
       'teaching_experience', 'internships'],
      dtype='object')
INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4817 entries, 0 to 4816
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   personal_info        4816 non-null   object
 1   experience           4817 non-null   object
 2   education            4817 non-null   object
 3   skills               4817 non-null   object
 4   projects             4806 non-null   object
 5   certifications       4817 non-null   object
 6   achievements         4 non-null      object
 7   workshops            4 non-null      object
 8   publications         3 non-null      object
 9   teaching_experience  1 non-null      object
 10  internships          2 non-null      object
dtypes: object(11)
memory usage: 414.1+ 

In [39]:

# Helper: safe getters
def safe_get(d, *keys):
    try:
        for k in keys:
            d = d[k]
        return d
    except Exception:
        return None

# 2) Extract name and email from personal_info (robust)
def extract_from_personal(info, key):
    if not isinstance(info, dict):
        return "Not Available"
    # common direct keys
    if key in info and info[key]:
        val = info[key]
        if isinstance(val, (list, tuple)) and len(val) > 0:
            return str(val[0]).strip()
        return str(val).strip()
    # try alternative keys
    alt_keys = {
        'name': ['fullname', 'full_name', 'displayname'],
        'email': ['e-mail', 'mail']
    }
    for alt in alt_keys.get(key, []):
        if alt in info and info[alt]:
            return str(info[alt]).strip()
    # fallback: search nested dict values for email pattern / likely name
    # quick pass: scan string values
    import re
    email_re = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")
    for v in info.values():
        if isinstance(v, str) and v.strip():
            if key == 'email' and email_re.search(v):
                return email_re.search(v).group(0)
            if key == 'name' and len(v.split()) <= 5 and not email_re.search(v):
                return v.strip()
        if isinstance(v, (list, tuple)):
            for item in v:
                if isinstance(item, str):
                    if key == 'email' and email_re.search(item):
                        return email_re.search(item).group(0)
                    if key == 'name' and len(item.split()) <= 5 and not email_re.search(item):
                        return item.strip()
    return "Not Available"

raw_df['name'] = raw_df['personal_info'].apply(lambda x: extract_from_personal(x, 'name') if 'personal_info' in raw_df.columns else "Not Available")
raw_df['email'] = raw_df['personal_info'].apply(lambda x: extract_from_personal(x, 'email') if 'personal_info' in raw_df.columns else "Not Available")

# 3) Extract education (take the first education entry's degree level + field if present)
def extract_education(edu):
    try:
        if isinstance(edu, list) and len(edu)>0 and isinstance(edu[0], dict):
            deg = edu[0].get('degree', {})
            if isinstance(deg, dict):
                level = deg.get('level') or deg.get('name') or ''
                field = deg.get('field') or ''
                if level or field:
                    return (str(level).strip() + (" in " + str(field).strip() if field else "")).strip()
            # fallback: look for 'institution' or 'college' or simple text
            inst = edu[0].get('institution') or edu[0].get('college') or edu[0].get('school')
            if inst:
                return str(inst)
        # if it's a string
        if isinstance(edu, str) and edu.strip():
            return edu.strip()
    except Exception:
        pass
    return "Not Available"

raw_df['education_clean'] = raw_df['education'].apply(extract_education)

# 4) Extract skills: walk nested dicts under 'skills' and collect 'name'/'skill' text or keys/values
def extract_skills(sk_dict):
    try:
        all_skills = []
        if isinstance(sk_dict, dict):
            # common pattern: {'technical': {'programming_languages': [{'name': 'Python'}, ...], ...}, 'languages': ...}
            def collect(x):
                if isinstance(x, dict):
                    # if dict has 'name' key and it's a string -> take it
                    if 'name' in x and isinstance(x['name'], str):
                        all_skills.append(x['name'].strip())
                    else:
                        for v in x.values():
                            collect(v)
                elif isinstance(x, list):
                    for it in x:
                        collect(it)
                elif isinstance(x, str):
                    all_skills.append(x.strip())
            collect(sk_dict)
        # dedupe and join
        all_skills = [s for s in pd.unique(all_skills) if s and s.lower() not in ('none','n/a','unknown')]
        return ', '.join(all_skills) if all_skills else "Not Available"
    except Exception:
        return "Not Available"

raw_df['skills_clean'] = raw_df['skills'].apply(extract_skills)

# 5) Extract project names (collect 'name' from each project entry)
def extract_projects(proj_list):
    try:
        if isinstance(proj_list, list):
            names = []
            for p in proj_list:
                if isinstance(p, dict):
                    if 'name' in p and p['name']:
                        names.append(str(p['name']).strip())
                    else:
                        # maybe title
                        if 'title' in p and p['title']:
                            names.append(str(p['title']).strip())
                elif isinstance(p, str):
                    names.append(p.strip())
            names = [n for n in pd.unique(names) if n and n.lower() not in ('none','n/a','unknown')]
            return ', '.join(names) if names else "Not Available"
        if isinstance(proj_list, str) and proj_list.strip():
            return proj_list.strip()
    except Exception:
        pass
    return "Not Available"

raw_df['projects_clean'] = raw_df['projects'].apply(extract_projects)

# 6) Extract top experience entry (title + company) or fallback text
def extract_experience(exp_list):
    try:
        if isinstance(exp_list, list) and len(exp_list)>0:
            e0 = exp_list[0]
            if isinstance(e0, dict):
                title = e0.get('title') or e0.get('designation') or ''
                company = e0.get('company') or e0.get('organization') or ''
                if title or company:
                    s = (str(title).strip() + (" at " + str(company).strip() if company else "")).strip()
                    return s if s else "Not Available"
            # fallback to text length
            if isinstance(e0, str) and e0.strip():
                return e0.strip()
        if isinstance(exp_list, str) and exp_list.strip():
            return exp_list.strip()
    except Exception:
        pass
    return "Not Available"

raw_df['experience_clean'] = raw_df['experience'].apply(extract_experience)

# 7) Build final dataframe with selected columns and rename
final_df = raw_df[['name','email','education_clean','skills_clean','experience_clean','projects_clean']].copy()
final_df.rename(columns={
    'education_clean': 'education',
    'skills_clean': 'skills',
    'experience_clean': 'experience',
    'projects_clean': 'projects'
}, inplace=True)

# 8) Drop rows with missing core skills (optional) - keep for now but report counts
print("\nCounts before dropping missing skills/education:")
print("Total records:", len(final_df))
print("Rows with skills = Not Available:", (final_df['skills']=='Not Available').sum())
print("Rows with education = Not Available:", (final_df['education']=='Not Available').sum())

# 9) Save cleaned CSV
final_df.to_csv("../DATA/resume_dataset_clean.csv", index=False)
print("\nSaved cleaned dataset to ../DATA/resume_dataset_clean.csv")

# 10) Show preview
print("\nPreview of cleaned data (first 8 rows):")
display(final_df.head(8))

# 11) Basic stats
print("\nUnique skills (sample):")
# quick sample of top skill tokens (not perfect tokenization, just a glance)
skills_series = final_df['skills'].replace("Not Available", pd.NA).dropna().str.split(', ')
top_skills = pd.Series([s for sub in skills_series for s in sub]).value_counts().head(15)
print(top_skills)


C:\Users\VAIBHAV VARSHNEY\AppData\Local\Temp\ipykernel_20128\3814422559.py:94: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  all_skills = [s for s in pd.unique(all_skills) if s and s.lower() not in ('none','n/a','unknown')]
C:\Users\VAIBHAV VARSHNEY\AppData\Local\Temp\ipykernel_20128\3814422559.py:116: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = [n for n in pd.unique(names) if n and n.lower() not in ('none','n/a','unknown')]



Counts before dropping missing skills/education:
Total records: 4817
Rows with skills = Not Available: 12
Rows with education = Not Available: 4

Saved cleaned dataset to ../DATA/resume_dataset_clean.csv

Preview of cleaned data (first 8 rows):


,name,email,education,skills,experience,projects
0,Unknown,Unknown,ME in Computer Engineering,"Python, C++, C, Tensorflow, Numpy, MySQL",Python Developer at Fresher,Not Available
1,Unknown,Unknown,B.E in Electronics,"Project Execution, Quality Management, Budget ...","Operations Manager at Delta Controls, Dubai FZCO","FGP/WPMP, Grain LNG RTLF, AMC TO HPM Migration..."
2,Not Provided,Not Provided,B.E. in Not Provided,"C, SQL, PL/SQL, JAVA, JAVAEE, Javascript, HTML...",DevOps Engineer at Parkar Consulting and Labs,"FPGA Implementation, Sorting Robot, RFID Based..."
3,Unknown,Unknown,B.E in Electronics,"Project Execution, Quality Management, Budget ...","Operations Manager at Delta Controls, Dubai FZCO","FGP/WPMP, Grain LNG RTLF, AMC TO HPM Migration..."
4,Not Available,Not Available,Bachelor of Engineering in Computer,"Python, Django, MySQL",Python Developer at Atos Syntel,Not Available
5,Not Available,Not Available,Diploma in Computer Science,Python,Python Developer at Cybage Software Pvt. Ltd,Not Available
6,Not Available,Not Available,B.E in Computer Science and Engineering,"C, C++, J2EE, Spring, Hibernate, My SQL, Postg...",Java Developer at Inouvelle Ventures Private L...,Stock Management System
7,Unknown,Unknown,High School in Science,"Java, Spring, Hibernate, MySQL",Java Developer at Kunal IT Services Pvt Ltd,Not Available



Unique skills (sample):
English         4644
Python          2793
JavaScript      1780
AWS             1698
Java            1597
Google Cloud    1589
React           1519
Azure           1484
MySQL           1453
MongoDB         1429
Redis           1410
Node.js         1398
Oracle          1397
Ruby            1384
C++             1384
Name: count, dtype: int64
